In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
#reading weather csv and making it a DF
city_file_path = "city_weather.csv"
city_weather_df = pd.read_csv(city_file_path)
#remove first column 
city_weather_df = city_weather_df.drop(city_weather_df.columns[0], axis=1)
#dropping duplicate cities
city_weather_df = city_weather_df.drop_duplicates(subset=["City"])
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hilo,19.7297,-155.0900,78.28,85,0,8.05,us,1655105692
1,icatu,-2.7758,-44.0658,79.23,93,0,3.47,br,1655105693
2,port alfred,-33.5906,26.8910,72.18,42,100,4.00,za,1655105450
3,albany,42.6001,-73.9662,65.39,93,100,2.53,au,1655105615
4,tabou,4.4230,-7.3528,78.84,72,88,7.25,ci,1655105697
...,...,...,...,...,...,...,...,...,...
1824,uray,60.1247,64.7758,52.23,79,100,11.32,ru,1655107922
1825,big bend,-26.8167,31.9333,70.83,24,0,5.12,sz,1655107923
1829,agadir,30.4202,-9.5982,70.25,76,72,2.39,ma,1655107928
1831,dagana,16.5165,-15.5068,74.43,80,91,12.64,sn,1655107930


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
locations_df = city_weather_df[["Lat", "Lng"]]
locations_df.head(5)

,Lat,Lng
0,19.7297,-155.0900
1,-2.7758,-44.0658
2,-33.5906,26.8910
3,42.6001,-73.9662
4,4.4230,-7.3528


In [58]:
#loading gmaps with my api key
gmaps.configure(api_key=google_api_key)
# delcaring variables, lat, long, and humidity

humidity_weight = city_weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_weather_df[["Lat", "Lng"]], weights=humidity_weight, 
                                 dissipating=False, max_intensity=600,
                                 point_radius=8)


# Customize the size of the figure
figure_layout = {
    'width': '2000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My weather criteria:

Temp between 20 and 70 (inclusive) - I like cold places i guess!
Wind Speed greater or equal to 10 MPH - Windly places too
Cloudiness a lot, greater to equal to 80 - and clouds!


In [63]:
#create new dataframe to play with / filter to match criteria

ideal_weather_df = city_weather_df


ideal_weather_df = city_weather_df[(city_weather_df["Max Temp"] >= 20) & (city_weather_df["Max Temp"] <=70) & (city_weather_df["Wind Speed"] >=10) & (city_weather_df["Cloudiness"] >=80)]
ideal_weather_df.head(5)

#adding hotel name as a column to DF
ideal_weather_df["Hotel Name"] = ""
ideal_weather_df


c:\Users\Dog\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,geraldton,-28.7667,114.6000,66.61,59,100,11.50,au,1655105478,
10,hermanus,-34.4187,19.2345,61.81,83,100,12.19,za,1655105704,
31,dikson,73.5069,80.5464,30.22,90,100,17.00,ru,1655105728,
32,sao joao da barra,-21.6403,-41.0511,65.48,64,100,19.17,br,1655105729,
35,busselton,-33.6500,115.3333,61.52,71,90,14.18,au,1655105733,
45,bluff,-46.6000,168.3333,46.80,63,80,27.36,nz,1655105746,
57,katsuura,35.1333,140.3000,66.72,69,100,13.56,jp,1655105763,
84,naryan-mar,67.6713,53.0870,62.82,33,85,10.18,ru,1655105798,
124,cidreira,-30.1811,-50.2056,51.10,76,100,14.09,br,1655105847,
139,kaitangata,-46.2817,169.8464,45.45,78,90,16.13,nz,1655105865,


In [64]:
#getting nearest hotel name from API, then appending it in the dataframe

google_api_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": "41.5284, -87.4237",  #just leftover lat lng cooridnates from lesson code
    #"rankby": "distance",  #can't use rankby distance if using radius as per api documentation :)
    "type": "lodging", #lodging is a hotel........
    "key": google_api_key,
    "radius" : "5000" # list of itels within 5000 meters of the location, as per the instructions!
}

for index, row in ideal_weather_df.iterrows():
    

    param_lat_lng = str(row["Lat"]) + ", " + str(row["Lng"])
    params["location"] = param_lat_lng

    try:
        response = requests.get(google_api_base_url, params=params).json()
   

        name_of_hotel = response["results"][0]["name"] #get the first result hotel
        ideal_weather_df.loc[index, 'Hotel Name'] = name_of_hotel # adding the name of the first hotel to the dataframe
    except:
            #basically any errors means there's no results
          ideal_weather_df.loc[index, 'Hotel Name'] = "No Hotel within 5000m"
   
    

    
ideal_weather_df

c:\Users\Dog\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,geraldton,-28.7667,114.6000,66.61,59,100,11.50,au,1655105478,Broadwater Mariner Resort
10,hermanus,-34.4187,19.2345,61.81,83,100,12.19,za,1655105704,Misty Waves Boutique Hotel
31,dikson,73.5069,80.5464,30.22,90,100,17.00,ru,1655105728,No Hotel within 5000m
32,sao joao da barra,-21.6403,-41.0511,65.48,64,100,19.17,br,1655105729,Pousada Porto De Canoas
35,busselton,-33.6500,115.3333,61.52,71,90,14.18,au,1655105733,Observatory Guest House
45,bluff,-46.6000,168.3333,46.80,63,80,27.36,nz,1655105746,Bluff Homestead - Guesthouse & Campervan Park
57,katsuura,35.1333,140.3000,66.72,69,100,13.56,jp,1655105763,Katsuura Hotel Mikazuki
84,naryan-mar,67.6713,53.0870,62.82,33,85,10.18,ru,1655105798,Hostel Agat
124,cidreira,-30.1811,-50.2056,51.10,76,100,14.09,br,1655105847,Hotel Castelo
139,kaitangata,-46.2817,169.8464,45.45,78,90,16.13,nz,1655105865,Kaitangata Motor Camp


In [68]:
# Create a map with markers

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations,display_info_box=True,info_box_content=hotel_info)
# Add layer
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…